<a href="https://colab.research.google.com/github/anushkamadiwale04/Face-Recognition-in-Python/blob/main/ecosystem_solver_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# INTERACTIVE ECOSYSTEM BUILDING TOOL

# This code defines an interactive tool for building a food chain within a specified ecosystem.
# Users can select the ecosystem (ocean or mountain), list food source names, set food source parameters,
# pick the most promising parameter range, list animal names, and set animal parameters.
# The tool then evaluates various combinations to find a sustainable food chain that meets the specified conditions.
# It uses interactive widgets and data visualization to guide users through the process.

# --------------- GAME RULES ---------------
# §1: Species with the highest calorie value consume first, followed by the second-highest, and so on.
# §2: While eating, species with the highest calorie value are consumed first.
# §3: In case of food source ties, required calories are divided equally among them.
# §4: Consuming reduces the calories provided by the same amount as the consuming species' needs.
# §5: If a food source is insufficient, the species eats the second-highest calorie food source.
#     Note: This rule should never lead to extinction as per important note.
# §6: A species survives if its required calories are met and provided calories are above zero.

# --------------- INSTRUCTIONS ---------------
# Run the first code block by clicking in it and pressing [ctrl]+[enter]
# Fill in the required information in the form which appears under the code block
# Continue like this for the other codeblocks. There are 4 input forms in total.

# --------------- AUTHOR ---------------
# Author: Matthias Standfest
# Date: Aug 10th 2023
# LinkedIn: https://www.linkedin.com/in/standfest/

# --------------- LICENSE ---------------
# The code of this notebook is subject to the terms of the AGPL v3 license.
# You can find the full license text at https://www.gnu.org/licenses/agpl-3.0.en.html

# --------------- REQUIREMENTS ---------------
# - ipydatagrid: A widget library for creating interactive data grids.
# - ipywidgets: Interactive HTML widgets for Jupyter notebooks.
# - seaborn: A data visualization library based on Matplotlib.
# - pandas: A data manipulation and analysis library.
# - itertools: A module for efficient looping and combinations.

import seaborn as sns # Visualization library
import pandas as pd # Data manipulation library
from itertools import combinations # Combinations generator
import copy  # For deep copying data structures
import ipywidgets as widgets  # Interactive widget library https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
from IPython.display import display, Javascript  # Display library function
from google.colab import output  # For custom widget management in Colab
output.enable_custom_widget_manager() # Enable custom widget manager
try:
    import ipydatagrid #https://github.com/bloomberg/ipydatagrid
except:
    !pip install -q ipydatagrid  # Install ipydatagrid on Colab server
    from ipydatagrid import DataGrid, TextRenderer, VegaExpr # Load libraries


# Constants for visual representation
CELL_BACKGROUND_COLORS = sns.color_palette("pastel").as_hex() #using a generic color palette for the outputs
USE_FIRST_SOLUTION_ONLY = True # If True, fastest result is provided; else, all versions are generated

# Game parameters
MIN_COUNT_FOODSOURCES = 9 #number of inputs provided by game interface
MIN_COUNT_ANIMALS = 10 #number of inputs provided by game interface
MIN_COUNT_SELECTED_ANIMALS = 5 # Minimum number of selected animals required for simulation and to pass the game

# Predefined demo data for quick testing and for functionality test
DEMO_FOODSOURCES_NAMES='Acro Coral;Brain Coral;Doughnut Coral;Giant Kelp;Hammer Coral;Red Coral;Red Seaweed;Seaweed;Stylophora'
DEMO_ANIMALS_NAMES='Bigeye Fish;Bigeye Tuna;Cod;Dotted Fish;Killer Whale;Lionfish;Mackarel;Turtle;Violet Fish;White Shark'
DEMO_ECOSYSTEM='ocean'
DEMO_FOODSOURCE_DATA = pd.DataFrame({
    "food_source":['Acro Coral','Brain Coral','Doughnut Coral','Giant Kelp','Hammer Coral','Red Coral','Red Seaweed','Seaweed','Stylophora'],
    "min_depth":[61,31,31,0,61,31,0,0,61],
    "max_depth":[90,60,60,30,90,60,30,30,90],
    "min_temp":[26.7,28.3,28.3,30.1,26.7,28.3,30.1,30.1,26.7],
    "max_temp":[28.2,30,30,31.5,28.2,30,31.5,31.5,28.2],
    "cal_provided":[4400,4600,4700,4150,4300,5050,5100,4800,4400],
    "is_poisonous":[0,0,0,0,0,0,0,0,0]}
    )
DEMO_ANIMAL_DATA = pd.DataFrame({
    "food_source":['Bigeye Fish','Bigeye Tuna','Cod','Dotted Fish','Killer Whale','Lionfish','Mackarel','Turtle','Violet Fish','White Shark'],
    "cal_provided":[3150,650,2100,2200,3100,1650,3650,1100,1500,1800],
    "cal_needed":[3200,1600,5600,4500,2200,2500,4300,2600,4200,1300],
    'Bigeye Fish':[0,	1,	0,	0,	0,	0,	0,	0,	0,	1],
    'Bigeye Tuna':[0,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Cod':[0,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Dotted Fish':[1,	0,	0,	0,	1,	1,	0,	0,	0,	0],
    'Killer Whale':[0,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Lionfish':[0,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Mackarel':[1,	0,	1,	0,	0,	0,	0,	0,	0,	0],
    'Turtle':[1,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Violet Fish':[0,	0,	1,	0,	1,	0,	0,	0,	0,	0],
    'White Shark':[0,	0,	0,	0,	0,	0,	0,	0,	0,	0],
    'Brain Coral':[0,	0,	0,	0,	0,	0,	1,	0,	0,	0],
    'Doughnut Coral':[0,	0,	0,	0,	0,	0,	1,	1,	1,	0],
    'Red Coral':[0,	0,	0,	1,	0,	0,	1,	0,	1,	0],
    }
    )
DEMO_FAILURE_SELECTION = [True,True,True,True,True,False,False,False,False,False] #test selection to generate failure
DEMO_SUCCESS_SELECTION = [True,True,False,False,False,True,False,False,True,True] #test selection to generate success

# --------------- MATHEMATICAL CONCEPT ---------------
# When drawing 5 out of 10 items without replacement, the number of different combinations
# can be calculated using the binomial coefficient formula, also known as "n choose k." The formula is given by:
#
# C(n, k) = n! / (k! * (n - k)!)
#
# n is the total number of items (10 in this case)
# k is the number of items to be drawn (5 in this case)
# n! represents the factorial of n (n * (n - 1) * (n - 2) * ... * 1)
# k! represents the factorial of k
# (n - k)! represents the factorial of (n - k)
# Using this formula, the number of different combinations when drawing 5 out of 10 without replacement is:
#
# C(10, 5) = 10! / (5! * (10 - 5)!) = 252
#
# So, there are 252 different combinations when drawing 5 out of 10 items without replacement.

# DECISION:
# While generally using DEAP to optimise tasks like this foodchain-building would be implemented in a production
# environment, loading the library would take more time than brute forcing this very specific case. Hence we can
# generate all the possible solutions upfront.
# ------------------------------------------------------

# Generate all combinations of drawing k items out of n
ALL_POSSIBLE_COMBINATIONS = list(combinations(range(MIN_COUNT_ANIMALS), MIN_COUNT_SELECTED_ANIMALS))


# GENERATE INTERACTIVE FORM 1 OF 4

screen1_ecosystem_picker_caption = widgets.Label(
    value='Which ecosystem to solve:'
    )

screen1_ecosystem_picker = widgets.RadioButtons(
    options=['ocean', 'mountain'],
    disabled=False
)

screen1_foodsources_namelist_caption = widgets.Label(
    value='The names of the foodsources (separated by semikolon):'
    )

screen1_foodsources_namelist = widgets.Textarea(
  placeholder='List the foodsource names separated by semikolon',
  disabled=False,
  layout=widgets.Layout(width='500px', height='80px')
)

screen1_use_demodata = widgets.Button(
    description='Use demo data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Use demo data',
    icon='clone', # (FontAwesome names without the `fa-` prefix)
)

screen1_status = widgets.Button(
    description='Check Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='500px')
)

# Display interactive elements on the screen
display(screen1_use_demodata,screen1_ecosystem_picker_caption, screen1_ecosystem_picker, screen1_foodsources_namelist_caption, screen1_foodsources_namelist, screen1_status)

# Callback function to populate fields with demo data
def on_click_screen1_use_demodata(e):
  screen1_foodsources_namelist.value=DEMO_FOODSOURCES_NAMES
  screen1_ecosystem_picker.value=DEMO_ECOSYSTEM
  screen1_status.description='Check Input'
  screen1_status.button_style='' # 'success', 'info', 'warning', 'danger' or ''
  screen1_status.icon='' # (FontAwesome names without the `fa-` prefix)

# Callback function to validate food source input and update status button
def on_click_screen1_status(e):
  if len(screen1_foodsources_namelist.value.split(';'))>=MIN_COUNT_FOODSOURCES:
    screen1_status.description='Food sources listed successfully. Run next Cell now.'
    screen1_status.button_style='success' # 'success', 'info', 'warning', 'danger' or ''
    screen1_status.icon='check' # (FontAwesome names without the `fa-` prefix)
  else:
    screen1_status.description='Not enough food sources listed! Add more.'
    screen1_status.button_style='danger' # 'success', 'info', 'warning', 'danger' or ''
    screen1_status.icon='times' # (FontAwesome names without the `fa-` prefix)

# Attach callback functions to button clicks
screen1_use_demodata.on_click(on_click_screen1_use_demodata)
screen1_status.on_click(on_click_screen1_status)

Button(description='Use demo data', icon='clone', style=ButtonStyle(), tooltip='Use demo data')

Label(value='Which ecosystem to solve:')

RadioButtons(options=('ocean', 'mountain'), value='ocean')

Label(value='The names of the foodsources (separated by semikolon):')

Textarea(value='', layout=Layout(height='80px', width='500px'), placeholder='List the foodsource names separat…

Button(description='Check Input', layout=Layout(width='500px'), style=ButtonStyle())

In [ ]:
# Split the input food source names separated by semicolon
foodsource_names = screen1_foodsources_namelist.value.split(';')

# Determine the altitude suffix based on ecosystem selection
altitude_suffix = "height" if screen1_ecosystem_picker.value == "mountain" else "depth"

# Create an empty DataFrame to store food source information
foodsource_empty = pd.DataFrame({
    "food_source":foodsource_names,
    "min_"+altitude_suffix:[0]*len(foodsource_names),
    "max_"+altitude_suffix:[0]*len(foodsource_names),
    "min_temp":[0]*len(foodsource_names),
    "max_temp":[0]*len(foodsource_names),
    "cal_provided":[0]*len(foodsource_names),
    "is_poisonous":[0]*len(foodsource_names)}
    )

# GENERATE INTERACTIVE FORM 2 OF 4

screen2_foodsource_input = DataGrid(
    foodsource_empty,
    selection_mode="cell",
    grid_style={"void_color": "#383838"},
    layout={"height": "320px"},
    base_row_size=32,
    base_column_size=150,
    editable=True
    )

screen2_use_demodata = widgets.Button(
    description='Use demo data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Use demo data',
    icon='clone', # (FontAwesome names without the `fa-` prefix)
)

screen2_status = widgets.Button(
    description='Run next cell once you have entered all the parameters.',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='500px')
)

screen2_foodsource_input_caption = widgets.Label(
    value='Fill in the missing parameters of the food sources:'
    )

screen2_status_message = widgets.HTML(
    value=' '
    )

# Display interactive elements on the screen
display(screen2_use_demodata,screen2_foodsource_input_caption,screen2_foodsource_input, screen2_status, screen2_status_message)

# Callback function to populate DataGrid with demo data
def on_click_screen2_use_demodata(e):
  screen2_foodsource_input.data = DEMO_FOODSOURCE_DATA
  screen2_status_message.value = ''
  screen2_status.description = 'Run next cell once you have entered all the parameters.'
  screen2_status.button_style=''

# Attach callback function to demo data button click
screen2_use_demodata.on_click(on_click_screen2_use_demodata)

Button(description='Use demo data', icon='clone', style=ButtonStyle(), tooltip='Use demo data')

Label(value='Fill in the missing parameters of the food sources:')

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=150, base_row_size=…

Button(description='Run next cell once you have entered all the parameters.', layout=Layout(width='500px'), st…

HTML(value=' ')

In [ ]:
# ECOSYSTEM IDENTIFICATION PROCESS
# identify the ecosystem paramaters which lead to the maximum cal for the foodchain

# Filter the foodsource DataFrame based on 'is_poisonous' condition (cannot be used for the foodchain)
useable_foodsources = screen2_foodsource_input.data[screen2_foodsource_input.data['is_poisonous'] == 0]

# Group by the specified columns and calculate the sum and count of 'cal_provided'
grouped_foodsources = useable_foodsources.groupby([
    "min_" + altitude_suffix,
    "max_" + altitude_suffix,
    "min_temp",
    "max_temp"
])['cal_provided'].agg(['sum', 'count'])

# Sort the grouped data by 'sum' column in descending order
sorted_grouped_foodsources = grouped_foodsources.sort_values(by='sum', ascending=False)

# Extract the best ecosystem parameters based on highest caloric value
best_ecosystem_parameters = sorted_grouped_foodsources.index[0]

# Define conditions for filtering food sources based on best parameters
condition_min_height = screen2_foodsource_input.data["min_"+altitude_suffix] == best_ecosystem_parameters[0]
condition_max_height = screen2_foodsource_input.data["max_"+altitude_suffix] == best_ecosystem_parameters[1]
condition_min_temp = screen2_foodsource_input.data["min_temp"] == best_ecosystem_parameters[2]
condition_max_temp = screen2_foodsource_input.data["max_temp"] == best_ecosystem_parameters[3]

# Combine the conditions using the '&' operator
combined_condition = (
    condition_min_height &
    condition_max_height &
    condition_min_temp &
    condition_max_temp
)

# Use .loc to filter the DataFrame based on the conditions and get the keys of the foodsources with the highest calories
best_foodsource_keys = screen2_foodsource_input.data.index[combined_condition].tolist()

# Define renderer to highlight rows corresponding to best food sources
highlight_best_foodsources = TextRenderer(
    background_color=VegaExpr(
        "cell.metadata.row == "+str(best_foodsource_keys[0])+" | cell.metadata.row == "+str(best_foodsource_keys[1])+" | cell.metadata.row == "+str(best_foodsource_keys[2])+" ? '"+CELL_BACKGROUND_COLORS[0]+"' : '#ffffff'"
    )
)

# Register renderer with the DataGrid
screen2_foodsource_input.default_renderer=highlight_best_foodsources

# Update status button to indicate successful ecosystem identification
screen2_status.description='Ecosystem successfully identified'
screen2_status.button_style='success' # 'success', 'info', 'warning', 'danger' or ''
screen2_status.tooltip='Ecosystem successfully identified'
screen2_status.icon='check' # (FontAwesome names without the `fa-` prefix)

# Calculate the sum of caloric values for best food sources
sum_cal_best_foodsources = sorted_grouped_foodsources.iloc[0,0]
environmental_parameters_best_foodsources = sorted_grouped_foodsources.index[0]

# Create status text message content
screen2_status_message_content = (
    f"Only pick animals with these environmental conditions:<br><ul>"
    f"<li>min_{altitude_suffix}: <b>{environmental_parameters_best_foodsources[0]}</b></li>"
    f"<li>max_{altitude_suffix}: <b>{environmental_parameters_best_foodsources[1]}</b></li>"
    f"<li>min_temp: <b>{environmental_parameters_best_foodsources[2]}</b></li>"
    f"<li>max_temp: <b>{environmental_parameters_best_foodsources[3]}</b></li>"
    f"</ul>"# this could be added here: In total, foodscources in this are can produce <u><b>{foodsource_cal_sum}</b></u> cal."
)

# Set caption to the status text message
screen2_status_message.value = screen2_status_message_content

# GENERATE INTERACTIVE FORM 3 OF 4

screen3_animals_namelist_caption = widgets.Label(
    value='The names of the animals of the chosen ecosystem (separated by semikolon):'
    )

screen3_animals_namelist = widgets.Textarea(
  placeholder='List the foodsource names separated by semikolon',
  disabled=False,
  layout=widgets.Layout(width='500px', height='80px')
)

screen3_use_demodata = widgets.Button(
    description='Use demo data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Use demo data',
    icon='clone', # (FontAwesome names without the `fa-` prefix)
)

screen3_status = widgets.Button(
    description='Check Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='500px')
)

# Display interactive elements on the screen
display(screen3_use_demodata, screen3_animals_namelist_caption, screen3_animals_namelist, screen2_status_message, screen3_status)

# Callback function to populate animal names with demo data
def on_click_screen3_use_demodata(e):
  screen3_animals_namelist.value=DEMO_ANIMALS_NAMES
  screen3_status.description='Check Input'
  screen3_status.button_style='' # 'success', 'info', 'warning', 'danger' or ''
  screen3_status.icon='' # (FontAwesome names without the `fa-` prefix)

# Callback function to check animal names input status
def on_click_screen3_status(e):
  if len(screen3_animals_namelist.value.split(';'))>=MIN_COUNT_ANIMALS:
    screen3_status.description='Animal names listed successfully. Run next Cell now.'
    screen3_status.button_style='success' # 'success', 'info', 'warning', 'danger' or ''
    screen3_status.icon='check' # (FontAwesome names without the `fa-` prefix)
  else:
    screen3_status.description='Not enough animal names listed! Add more.'
    screen3_status.button_style='danger' # 'success', 'info', 'warning', 'danger' or ''
    screen3_status.icon='times' # (FontAwesome names without the `fa-` prefix)

# Attach callback functions to buttons
screen3_use_demodata.on_click(on_click_screen3_use_demodata)
screen3_status.on_click(on_click_screen3_status)

Button(description='Use demo data', icon='clone', style=ButtonStyle(), tooltip='Use demo data')

Label(value='The names of the animals of the chosen ecosystem (separated by semikolon):')

Textarea(value='', layout=Layout(height='80px', width='500px'), placeholder='List the foodsource names separat…

HTML(value='Only pick animals with these environmental conditions:<br><ul><li>min_depth: <b>31</b></li><li>max…

Button(description='Check Input', layout=Layout(width='500px'), style=ButtonStyle())

In [ ]:
# ANIMAL PARAMETERS INPUT FOR INTERACTIVE ELEMENTS

# Split the entered animal names by semicolon
animal_names = screen3_animals_namelist.value.split(';')

# Get names of selected food sources for the best ecosystem
selected_foodsource_names = screen2_foodsource_input.data["food_source"][best_foodsource_keys].tolist()

# Combine animal and food source names to get all species names
all_species_names = animal_names + selected_foodsource_names

# Create an empty DataFrame to capture animal parameters
animals_empty = pd.DataFrame({
    "food_source":animal_names,
    "cal_provided":[0]*len(animal_names),
    "cal_needed":[0]*len(animal_names)}
    )

# Add columns for all species names with initial values as 0
animals_empty[all_species_names]=[0]*len(all_species_names)

# Define rendering style for the species table
species_table_design = TextRenderer(
    horizontal_alignment=VegaExpr(
        "cell.metadata.column > 2  ? 'center' : 'left'"
    ),
    background_color=VegaExpr(
        "cell.metadata.column >12 ? '"+CELL_BACKGROUND_COLORS[7]+"' : '#ffffff'"
    ))

# GENERATE INTERACTIVE FORM 4 OF 4

screen4_animalparameters = DataGrid(
    animals_empty,
    selection_mode="cell",
    grid_style={"void_color": "#383838"},
    layout={"height": "350px"},
    default_renderer=species_table_design,
    base_row_size=32,
    base_column_size=100,
    editable=True
    )

screen4_use_demodata = widgets.Button(
    description='Use demo data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Use demo data',
    icon='clone', # (FontAwesome names without the `fa-` prefix)
)

screen4_status = widgets.Button(
    description='Run next cell once you have entered all the parameters.',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='500px')
)

screen4_animalparameters_caption = widgets.Label(
    value='Fill in the missing parameters (calories and which species serve as foodsources) of these species:'
    )

screen4_status_message = widgets.HTML(
    value=' '
    )

# Display interactive elements on the screen
display(screen4_use_demodata,screen4_animalparameters_caption,screen4_animalparameters, screen4_status, screen4_status_message)

# Callback function to populate animal parameters with demo data
def on_click_screen4_use_demodata(e):
  screen4_animalparameters.data = DEMO_ANIMAL_DATA
  screen4_status_message.value = ''
  screen4_status.description = 'Run next cell once you have entered all the parameters.'
  screen4_status.button_style=''

# Attach callback function to the demo data button
screen4_use_demodata.on_click(on_click_screen4_use_demodata)

Button(description='Use demo data', icon='clone', style=ButtonStyle(), tooltip='Use demo data')

Label(value='Fill in the missing parameters (calories and which species serve as foodsources) of these species…

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=100, base_row_size=…

Button(description='Run next cell once you have entered all the parameters.', layout=Layout(width='500px'), st…

HTML(value=' ')

In [ ]:
# FOODCHAIN IDENTIFICATION

# Sort animal parameters data based on 'cal_provided' column in descending order
sorted_animalparameters = screen4_animalparameters.data.sort_values(by=['cal_provided'], ascending=False)

# Select relevant columns for creating the adjacency matrix
adjacency_matrix_columns = sorted_animalparameters.columns.difference(["food_source", "cal_provided", "cal_needed"])

# Create a dictionary to store all species information
all_species = {} # Dictionary containing species details and their relationships for nested processing
for _, row in sorted_animalparameters.iterrows():
    foodsource = row["food_source"]
    own_foodsources = [adj_col for adj_col in adjacency_matrix_columns if row[adj_col] == 1]
    all_species[foodsource] = {"cal_provided":row["cal_provided"], "cal_needed":row["cal_needed"],"food_source": own_foodsources}

# Add selected food sources from the previous step to the species dictionary
for k in best_foodsource_keys:
  all_species[screen2_foodsource_input.data["food_source"][k]] = {"cal_provided":screen2_foodsource_input.data["cal_provided"][k], "cal_needed":[],"food_source": []}

# Function to evaluate the sustainability of species selection
def evaluate_species_selection(selection):
  """
  Evaluates the sustainability of a species selection for constructing a foodchain.

  This function simulates the interaction of selected species and their food sources to
  determine if the chosen combination can form a sustainable foodchain.

  Parameters:
      selection (list of bool): A list of boolean values indicating which species are selected.

  Returns:
      bool: True if the selection forms a sustainable foodchain, False otherwise.
  """

  # Initialize return variable
  is_sustainable = True

  # Create a deep copy of the species dictionary for simulation
  sim_species = copy.deepcopy(all_species)

  # Add plants to the selection
  selection = selection +[1,1,1] # Adding the 3 true values in the end, cause the plants are always picked

  # Create a list of keys to remove from selection
  keys = [list(sim_species)[index] for index, value in enumerate(selection) if value == False]

  # Remove species not part of the selection
  [sim_species.pop(key) for key in keys]

  # Iterate over selected species
  for name, species_iterator in sim_species.items():

    # List the individual food sources
    food = {key: sim_species[key] for key in species_iterator['food_source'] if key in sim_species}

    #if food is required
    if len(food)>0:

      # Calculate available calories from food sources
      calories_left = sum([v['cal_provided'] for v in food.values()])

      if calories_left > 0:

        # Check if there is a tie in highest calorie value among food sources
        cals = [v['cal_provided'] for v in food.values()]
        cmax = max(cals)
        result = [key for key, value in food.items() if value['cal_provided'] == cmax]
        count = len(result)

        # Handle the case of a tie
        if count >1:
          if species_iterator['cal_needed']<(cmax/count):
            # Subtract cal_needed divided by food_sources from cal_provided
            for r in result:
              sim_species[r]['cal_provided']-=(species_iterator['cal_needed']/count)
          else:
            is_sustainable = False
            break
        # Handle the case of not a tie (standard)
        else:
          if species_iterator['cal_needed']<cmax:
            # Subtract cal_needed from cal_provided
            sim_species[result[0]]['cal_provided']-=species_iterator['cal_needed']
          else:
            is_sustainable = False
            break
      else:
        is_sustainable = False
        break
    else:
      if len(species_iterator['food_source'])>0:
        is_sustainable = False
        break
  return is_sustainable

#test = evaluate_selection(test_selection_success)
#print(test)

# Generate a list of boolean combinations for species selection
boolean_selection_combinations = []
for combo in ALL_POSSIBLE_COMBINATIONS:
    boolean_combo = [False] * MIN_COUNT_ANIMALS
    for index in combo:
        boolean_combo[index] = True
    boolean_selection_combinations.append(boolean_combo)

# Sort boolean_combinations based on the sum of "cal_needed"
sorted_selection_combinations = sorted(boolean_selection_combinations, key=lambda combo: sum(sorted_animalparameters.loc[combo, "cal_needed"]))

# Iterate through sorted combinations and evaluate the foodchain
foodchain_identification_success = False #start condition of identification process
for combo in sorted_selection_combinations:
  if evaluate_species_selection(combo):
    selected_animals = sorted_animalparameters.loc[combo, "food_source"]
    selected_plants = screen2_foodsource_input.data["food_source"][best_foodsource_keys]

    # Reformat status button
    screen4_status.description='Foodchain identified successfully'
    screen4_status.button_style='success' # 'success', 'info', 'warning', 'danger' or ''
    screen4_status.icon='check' # (FontAwesome names without the `fa-` prefix)

    # Define renderer highlighting the chosen animals
    vega_string = " | ".join(["cell.metadata.data['food_source'] == '"+name+"'" for name in selected_animals.values])
    highlight_selected_animals = TextRenderer(
        horizontal_alignment=VegaExpr("cell.metadata.column > 2  ? 'center' : 'left'"),
        background_color=VegaExpr(
            vega_string+" ? '"+CELL_BACKGROUND_COLORS[0]+"' : '#ffffff'" #filter a line according to a value
        )
    )

    #Register renderer with the datagrid
    screen4_animalparameters.default_renderer=highlight_selected_animals

    animals_li=" ".join(["<li>"+name+"</li>" for name in selected_animals.values])
    plants_li=" ".join(["<li>"+name+"</li>" for name in selected_plants.values])
    statustext_foodchain = (
        f"Congratulations, you have successfully built a foodchain!<br>Just select these animals:<ol>{animals_li}</ol>"
        f"And select these food sources:<ol>{plants_li}</ol>"
        f"And select a location within these environmental parameters:<ol>"
        f"<li>min_{altitude_suffix}: <b>{environmental_parameters_best_foodsources[0]}</b></li>"
        f"<li>max_{altitude_suffix}: <b>{environmental_parameters_best_foodsources[1]}</b></li>"
        f"<li>min_temp: <b>{environmental_parameters_best_foodsources[2]}</b></li>"
        f"<li>max_temp: <b>{environmental_parameters_best_foodsources[3]}</b></li></ol>"
    )

    # Set caption to status text
    screen4_status_message.value = statustext_foodchain
    screen4_status_message.layout= widgets.Layout(top='0px', right='0px', width='500px')

    #Set success variable
    foodchain_identification_success = True

    # Exit loop if using only the first successful solution
    if USE_FIRST_SOLUTION_ONLY:
      break;

# Handle the case when foodchain identification fails
if not foodchain_identification_success:
    #reformat status button
    screen4_status.description='ERROR: Foodchain building failed'
    screen4_status.button_style='danger' # 'success', 'info', 'warning', 'danger' or ''
    screen4_status.icon='' # (FontAwesome names without the `fa-` prefix)

    statustext_foodchain = (
        f"It was not possible to identify a viable foodchain under the set conditions."
    )

    #set caption to status text
    screen4_status_message.value = statustext_foodchain